In [42]:
# Merge all recorded data

from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import helper_functions
%matplotlib inline


pd.set_option('mode.chained_assignment', None)

# Set up figure size and DPI for screen demo
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 150


# DL1	Stone Garden - Tjaden Hall
# DL2	PSB Parking Lot
# DL3	Olin Library
# DL4	Milstein Hall Roof
# DL5	Game Farm Road


S1 = HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv")
S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
S3 = HoboReader("DL3_Olin_Library.csv")
S4 = HoboReader("DL4_Milstein_Hall_Roof.csv")
S5 = HoboReader("DL5_Game_Farm_Road_20192020v5.csv")


df1 = S1.get_dataframe()
df2 = S2.get_dataframe()
df3 = S3.get_dataframe()
df4 = S4.get_dataframe()
df5 = S5.get_dataframe()

# DF5 (GAME FARM ROAD) HAS TO BE SHIFTED BACK BECAUSE IT NATIVELY RECORDS DST, OUR STATIONS DO NOT.
# THE OTHER STATIONS ARE NOT SHIFTED (HENCE 0 VALUE FOR PERIODS), JUST THERE FOR SANITY

df5 = df5.shift(periods=(-2), fill_value=0)
df4 = df4.shift(periods=(0), fill_value=0)
df3 = df3.shift(periods=(0), fill_value=0)
df2 = df2.shift(periods=(0), fill_value=0)
df1 = df1.shift(periods=(0), fill_value=0)

# CONVERT IMPERIAL TO METRIC UNITS REMOVED 9/3 - all data natively in METRIC

In [43]:
def f(variable_id, period_len, day, resample_len, month, year):
    
    start_time = str(year) + '-' + str(month) + '-' + str(        day) + ' 07:00:00-00:00'
    end_time = str(year) + '-' + str(month) + '-' + str(day+period_len-1) + ' 23:00:00-00:00'
    
    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]
    
#Resample subhourly data to 1 hourly graph (OUR STATIONS RECORD IN 10MIN INTERVALS, GFR IN 1HR, RESAMPLE FOR EASY COMPARISON. CAN TURN OFF)
    df1_ = df1_.resample(str(resample_len) + 'H').mean()
    df2_ = df2_.resample(str(resample_len) + 'H').mean()
    df3_ = df3_.resample(str(resample_len) + 'H').mean()
    df4_ = df4_.resample(str(resample_len) + 'H').mean()
    df5_ = df5_.resample(str(resample_len) + 'H').mean()
    
    # Translate to °C - REMOVED 9/3 - all data natively in METRIC

    #df1_["Temp"] = df1_["Temp"].apply(lambda x: ftoc(x), axis=1)
    #df2_["Temp"] = df2_["Temp"].apply(lambda x: ftoc(x), axis=1)
    #df3_["Temp"] = df3_["Temp"].apply(lambda x: ftoc(x), axis=1)
    #df4_["Temp"] = df4_["Temp"].apply(lambda x: ftoc(x), axis=1)

    #df1_["MRT"] = df1_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df2_["MRT"] = df2_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df3_["MRT"] = df3_["MRT"].apply(lambda x: ftoc(x), axis=1)
    #df4_["MRT"] = df4_["MRT"].apply(lambda x: ftoc(x), axis=1)

    # Translate to m/s

    #df1_["Wind Speed"] = df1_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df2_["Wind Speed"] = df2_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df3_["Wind Speed"] = df3_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df4_["Wind Speed"] = df4_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Wind Speed"] = df5_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)

    #df1_["Gust Speed"] = df1_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df2_["Gust Speed"] = df2_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df3_["Gust Speed"] = df3_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df4_["Gust Speed"] = df4_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Gust Speed"] = df5_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)

    # df1.index

    # Print Headers
    # print(df1.columns.values)

    # Resize figure
    custom_figsize = (30, 15)

    # Change lineweights
    lineweights = 1.5

    variables = [
        "Wind Speed", "MRT", "Temp", "RH", "Solar Radiation", "Wind Speed",
        "Gust Speed", "Wind Direction", "DewPt"
    ]

    variable = variables[variable_id]

    # Plot both temp and mrt when temp is selected

    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed":
        ax = df1_.plot(y=variable, lw=1, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=3.5,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y=variable, lw=2, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=3.5,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

        ax = df1_.plot(y="MRT",
                       lw=1,
                       figsize=custom_figsize,
                       style=['--'],
                       color='red',
                       ax=ax)
        df2_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='blue',
                  ax=ax)
        df3_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='green',
                  ax=ax)
        df4_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='aqua',
                  ax=ax)

    else:
        # Plot two dataframes into one plot
        ax = df1_.plot(y=variable, lw=1, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

    # Legend
    ax.legend([
        "DL1 - Stone Garden - Tjaden Hall", "DL2 - PSB Parking Lot",
        "DL3 - Olin Library", "DL4 - Milstein Hall Roof ",
        "DL5 - Game Farm Road"
    ],
              loc='upper left',
              frameon=False,fontsize=20)

    # Fix xticks

    #years = mdates.YearLocator()  # every year
    #months = mdates.MonthLocator()  # every month
    #days = mdates.DayLocator()  # every day
    
    #minutes = mdates.MinuteLocator()  # every minute

    #days_fmt = mdates.DateFormatter('%d')
    
    

    # format the ticks
    if period_len > 1:
        hours = mdates.HourLocator(interval = 6)  # every 6 hours
        hours_fmt = mdates.DateFormatter('%H:00')
        ax.xaxis.set_major_locator(hours)    
        ax.xaxis.set_major_formatter(hours_fmt)
        

    else:
        hours = mdates.HourLocator()  # every 6 hours
        hours_fmt = mdates.DateFormatter('%H:00')       
        
        ax.xaxis.set_major_locator(hours)    
        ax.xaxis.set_major_formatter(hours_fmt)
        
    # we dont have minor ticks for now
    #ax.xaxis.set_minor_locator(hours)
    #ax.tick_params(axis='both', which='minor', labelsize=22)
    ax.tick_params(axis='both', which='major', labelsize=14)
    
    

 
          

    # Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=26)
    plt.xlabel("UTC Time", fontname="Consolas", fontsize=26)
    

    # Title
    ax.set_title(str(
        start_time.split(' ')[0] + " "
        'to'
        " " + end_time.split(' ')[0] + " " + variable),
                 fontname="Consolas",
                 fontsize=28)

    # Save PDF
   # plot_graph(plt, start_time, variable)


# Variable to plot
# 1 = "MRT °C"
# 2 = "Temp °C"
# 3 = "RH"
# 4 = "Solar Radiation"
# 5 = "Wind Speed"
# 6 = "Gust Speed"
# 7 = "Wind Direction"
# 8 = "DewPt"

interact(f,  variable_id=(1, 8, 1),period_len=(1,30,1), resample_len=(1,24,1), day=(1, 31, 1), month=(1, 12, 1), year=(2019, 2021, 1))

interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=15, descrip…

<function __main__.f(variable_id, period_len, day, resample_len, month, year)>

In [6]:
#PRINT RESAMPLED DATA (HOURLY) TO A .CSV FOR EPW FILE

from pprint import pprint
import pandas as pd


S1 = HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv")
S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
S3 = HoboReader("DL3_Olin_Library.csv")
S4 = HoboReader("DL4_Milstein_Hall_Roof.csv")

df1 = S1.get_dataframe()
df2 = S2.get_dataframe()
df3 = S3.get_dataframe()
df4 = S4.get_dataframe()


df1 = df1.resample('H').mean()
df2 = df2.resample('H').mean()
df3 = df3.resample('H').mean()
df4 = df4.resample('H').mean()

display(df1)
display(df2)
display(df3)
display(df4)


print(df1)
print(df2)
print(df3)
print(df4)



df1.to_csv('DL1_TJADEN_hourly.csv')
df2.to_csv('DL2_PSB_hourly.csv')
df3.to_csv('DL3_OLIN_hourly.csv')
df4.to_csv('DL4_MILSTEIN_hourly.csv')




title,Pressure,Temp,RH,Solar Radiation,MRT,Wind Speed,Gust Speed,Wind Direction,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,in Hg,Â°C,%,W/mÂ²,Â°c,m/s,m/S,Ã¸,Â°C
logger_serial_number,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500
sensor_serial_number,10751992,10763331,10763331,10775592,20496099,20706789,20706789,20706789,10763331
datetimes,,,,,,,,,
2019-09-17 17:00:00-05:00,29.341467,19.428333,55.166667,3.966667,19.404630,1.001370,2.226259,112.600000,10.203704
2019-09-17 18:00:00-05:00,29.352800,18.029444,56.333333,0.600000,17.637037,0.724205,1.671930,61.466667,9.185185
2019-09-17 19:00:00-05:00,29.361167,16.105185,66.033333,0.600000,15.620278,0.275675,0.947725,53.033333,9.768519
2019-09-17 20:00:00-05:00,29.383333,14.576574,73.133333,0.600000,14.133704,0.277165,0.780830,227.600000,9.824074
2019-09-17 21:00:00-05:00,29.401517,13.372870,80.533333,0.600000,12.943333,0.275675,0.669070,234.633333,10.101852


title,Wind Speed,Gust Speed,Wind Direction,MRT,Temp,RH,Solar Radiation,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,m/s,m/s,Ã¸,Â°C,Â°C,%,W/mÂ²,Â°C
logger_serial_number,20702489,20702489,20702489,20702489,20702489,20702489,20702489,20702489
sensor_serial_number,20496534,20496534,20496534,20693834,20698453,20698453,20702166,20698453
datetimes,,,,,,,,
2019-09-28 09:00:00-05:00,1.671930,4.009949,184.400000,31.178889,24.797778,57.700000,1276.900000,15.888889
2019-09-28 10:00:00-05:00,1.280770,3.616554,231.833333,33.871204,26.324630,54.450000,584.183333,16.388889
2019-09-28 11:00:00-05:00,1.284495,3.507774,237.450000,37.643889,28.447222,49.966667,733.116667,17.018519
2019-09-28 12:00:00-05:00,1.451390,3.227629,36.583333,36.169074,29.387593,45.550000,480.416667,16.398148
2019-09-28 13:00:00-05:00,1.284495,2.783569,63.600000,33.850000,28.846481,47.116667,351.883333,16.490741


title,MRT,Temp,RH,Solar Radiation,Wind Speed,Gust Speed,Wind Direction,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,Â°C,Â°C,%,W/mÂ²,m/s,m/s,Ã¸,Â°C
logger_serial_number,20702490,20702490,20702490,20702490,20702490,20702490,20702490,20702490
sensor_serial_number,20496098,20698454,20698454,20702165,20706788,20706788,20706788,20698454
datetimes,,,,,,,,
2019-09-27 13:00:00-05:00,27.604444,22.405556,39.900000,691.900000,2.004974,5.009083,180.950000,8.083333
2019-09-27 14:00:00-05:00,29.302315,23.516944,38.400000,323.133333,1.339630,3.394524,325.450000,8.546296
2019-09-27 15:00:00-05:00,29.081111,24.074630,35.200000,349.783333,1.114620,3.007089,116.616667,7.777778
2019-09-27 16:00:00-05:00,24.844167,22.836481,39.083333,181.883333,1.114620,3.171004,234.150000,8.194444
2019-09-27 17:00:00-05:00,20.718981,20.781759,45.066667,15.000000,1.001370,2.670319,129.050000,8.425926


title,Solar Radiation,Temp,RH,Wind Speed,Gust Speed,Wind Direction,Water Content,MRT,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,W/mÂ²,Â°C,%,m/s,m/s,Ã¸,mÂ³/mÂ³,Â°C,Â°C
logger_serial_number,20094090,20094090,20094090,20094090,20094090,20094090,20094090,20094090,20094090
sensor_serial_number,20073988,20496044,20496044,20496535,20496535,20496535,20499689,20693833,20496044
datetimes,,,,,,,,,
2019-10-10 13:00:00-05:00,542.700000,17.668704,51.366667,1.114620,2.560049,250.833333,0.057033,24.180648,7.518519
2019-10-10 14:00:00-05:00,433.533333,18.140185,53.033333,1.171245,2.671809,271.000000,0.060233,23.978148,8.444444
2019-10-10 15:00:00-05:00,291.666667,17.811296,54.883333,1.339630,3.228374,293.300000,0.060917,23.008981,8.620370
2019-10-10 16:00:00-05:00,138.533333,16.879722,58.466667,1.451390,4.062103,286.033333,0.059750,20.623148,8.694444
2019-10-10 17:00:00-05:00,12.916667,15.237870,63.650000,0.891100,2.340254,271.000000,0.059250,15.240648,8.388889


title                       Pressure       Temp         RH Solar Radiation  \
timezone_str                     NaN        NaN        NaN             NaN   
units                          in Hg        Â°C          %           W/mÂ²   
logger_serial_number        20098500   20098500   20098500        20098500   
sensor_serial_number        10751992   10763331   10763331        10775592   
datetimes                                                                    
2019-09-17 17:00:00-05:00  29.341467  19.428333  55.166667        3.966667   
2019-09-17 18:00:00-05:00  29.352800  18.029444  56.333333        0.600000   
2019-09-17 19:00:00-05:00  29.361167  16.105185  66.033333        0.600000   
2019-09-17 20:00:00-05:00  29.383333  14.576574  73.133333        0.600000   
2019-09-17 21:00:00-05:00  29.401517  13.372870  80.533333        0.600000   
...                              ...        ...        ...             ...   
2020-09-04 09:00:00-05:00  29.129400  20.087167  70.083333      

In [44]:
# Reindl split test
import time
import math
import pandas as pd
from hoboreader import HoboReader
from gen_reindl import *

#S2 = HoboReader("DL2_PSB_hourly.csv")
df2 = S2.get_dataframe()

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods=8760, freq='H')
# print(date_series)

# altitude M Station (although not necessary for gen_reindl)
altitude = 255

# lat, lon taken from STATION
lon = 42.450536  # north-positive
lat = -76.481194  # west-negative #west of england

# pos
time_zone = 5 * 15  # west-positive

#print("month, day, time, GHR\n")

%timeit GenReindl(lat, lon, time_zone).calc_split(4, 22, 8.333333333, 107)

9.49 µs ± 851 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
#TJADEN HALL STATION

import time
from tqdm import tqdm

#S1 = HoboReader("DL1_TJADEN_2019.csv")
df2 = S2.get_dataframe()

def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0

    i = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        GHR = df.iloc[i, df.columns.get_loc('Solar Radiation')].values[0]
        DNI, DHR = GenReindl(lat, lon, time_zone).calc_split(df.month[i], df.day[i], df.hour[i], GHR)
        #print(i,  GHR, DNI, DHR)
        df.iloc[i, df.columns.get_loc('DHR')] = DHR
        df.iloc[i, df.columns.get_loc('DNI')] = DNI
        i += 1

    return df


df2 = calc_split_df(df2)


  2%|█▏                                                                            | 822/53708 [00:14<15:33, 56.64it/s]


  3%|██▍                                                                          | 1662/53708 [00:29<15:40, 55.33it/s]


  5%|███▌                                                                         | 2477/53708 [00:45<15:05, 56.57it/s]


  6%|████▋                                                                        | 3304/53708 [01:00<15:49, 53.08it/s]


  8%|█████▉                                                                       | 4126/53708 [01:16<15:48, 52.25it/s]


...

 78%|███████████████████████████████████████████████████████████▍                | 42041/53708 [12:43<03:18, 58.82it/s]


 80%|████████████████████████████████████████████████████████████▋               | 42875/53708 [12:58<03:23, 53.22it/s]


 81%|█████████████████████████████████████████████████████████████▊              | 43712/53708 [13:14<03:03, 54.33it/s]


 82%|██████████████████████████████████████████████████████████████▌             | 44184/53708 [13:22<02:44, 57.89it/s]

KeyboardInterrupt: 

In [21]:
# TJADEN HALL STATION

import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

#S5 = HoboReader("DL1_TJADEN_2019.csv")
#df1 = S1.get_dataframe()

# Template

template = epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns

# Real file

ithaca_2019 = epw()
# ithaca_2019.headers['LOCATION'][0]='Ithaca_NY'

#Design conditions and extreme periods are taken from the Syracuse file

ithaca_2019.headers = {'LOCATION': ['Ithaca',
                                    'NY',
                                    'USA',
                                    'TMY3',
                                    '725190',
                                    '42.438934',
                                    '-76.492435',
                                    '-5.0',
                                    '125.0'],
                       'DESIGN CONDITIONS': ['1',
  'Climate Design Data 2009 ASHRAE Handbook',
  '',
  'Heating',
  '1',
  '-19.3',
  '-16.2',
  '-23.6',
  '0.5',
  '-18.1',
  '-20.6',
  '0.6',
  '-15.1',
  '12.8',
  '-5.9',
  '11.9',
  '-5.9',
  '3',
  '90',
  'Cooling',
  '7',
  '10.8',
  '31.6',
  '22.8',
  '30',
  '21.8',
  '28.5',
  '21',
  '24.1',
  '29.3',
  '23.1',
  '27.9',
  '22.3',
  '26.8',
  '4.4',
  '260',
  '22.5',
  '17.4',
  '26.9',
  '21.5',
  '16.4',
  '25.8',
  '20.7',
  '15.6',
  '25.1',
  '72.9',
  '29.6',
  '69.2',
  '27.9',
  '65.9',
  '26.8',
  '695',
  'Extremes',
  '10.8',
  '9.1',
  '8.2',
  '27.6',
  '-23.6',
  '33.8',
  '4',
  '2',
  '-26.5',
  '35.2',
  '-28.8',
  '36.3',
  '-31.1',
  '37.4',
  '-34',
  '38.9'],
 'TYPICAL/EXTREME PERIODS': ['6',
  'Summer - Week Nearest Max Temperature For Period',
  'Extreme',
  '7/27',
  '8/ 2',
  'Summer - Week Nearest Average Temperature For Period',
  'Typical',
  '8/24',
  '8/30',
  'Winter - Week Nearest Min Temperature For Period',
  'Extreme',
  '1/27',
  '2/ 2',
  'Winter - Week Nearest Average Temperature For Period',
  'Typical',
  '12/22',
  '12/28',
  'Autumn - Week Nearest Average Temperature For Period',
  'Typical',
  '10/20',
  '10/26',
  'Spring - Week Nearest Average Temperature For Period',
  'Typical',
  '3/15',
  '3/21'],
                       'GROUND TEMPERATURES': ['3',
                                               '0.5',
                                               '',
                                               '',
                                               '',
                                               '2.44',
                                               '4.62',
                                               '9.34',
                                               '15.56',
                                               '21.51',
                                               '25.58',
                                               '26.67',
                                               '24.44',
                                               '19.55',
                                               '13.31',
                                               '7.40',
                                               '3.42',
                                               '2',
                                               '',
                                               '',
                                               '',
                                               '5.55',
                                               '6.08',
                                               '8.78',
                                               '13.07',
                                               '17.76',
                                               '21.58',
                                               '23.50',
                                               '23.98',
                                               '20.18',
                                               '15.85',
                                               '11.17',
                                               '7.40',
                                               '4',
                                               '',
                                               '',
                                               '',
                                               '8.79',
                                               '8.41',
                                               '9.60',
                                               '12.12',
                                               '15.30',
                                               '18.27',
                                               '20.23',
                                               '20.66',
                                               '19.42',
                                               '16.85',
                                               '13.67',
                                               '10.73'],
                       'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
                       'COMMENTS 1': [''],
                       'COMMENTS 2': [''],
                       'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

#print(ithaca_2019.headers)


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

ithaca_2019.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
ithaca_2019.dataframe['Month'] = template.dataframe['Month'].astype(int)
ithaca_2019.dataframe['Day'] = template.dataframe['Day'].astype(int)
ithaca_2019.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
ithaca_2019.dataframe['Minute'] = template.dataframe['Minute'].astype(int)
ithaca_2019.dataframe['Data Source and Uncertainty Flags'] = np.array(
    np.zeros(8760)).astype(int)


# Actual file starts here
# 6
# .apply(lambda x: ftoc(x), axis=1).values.flatten()
ithaca_2019.dataframe['Dry Bulb Temperature'] = df1['Temp'].values.flatten()

#     Add this later (ADDED TO DL5_2019 FILE 07.22.2020)
# np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Dew Point Temperature'] = df1['DewPoint'].values.flatten()


ithaca_2019.dataframe['Relative Humidity'] = df1['RH'].values.flatten()

# Can use our station pressure for this
ithaca_2019.dataframe['Atmospheric Station Pressure'] = df1['Pressure'].values.flatten()
ithaca_2019.dataframe['Extraterrestrial Horizontal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Extraterrestrial Direct Normal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Horizontal Infrared Radiation Intensity'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Global Horizontal Radiation'] = df1['Solar Radiation'].values.flatten()
ithaca_2019.dataframe['Direct Normal Radiation'] = df1['DNI'].values.flatten()
ithaca_2019.dataframe['Diffuse Horizontal Radiation'] = df1['DHR'].values.flatten()

# Do we need this?
# https://www.radiance-online.org//pipermail/radiance-general/2013-January/009140.html ?
ithaca_2019.dataframe['Global Horizontal Illuminance'] = df1['Solar Radiation'].values.flatten()*179
ithaca_2019.dataframe['Direct Normal Illuminance'] = df1['DNI'].values.flatten()*179
ithaca_2019.dataframe['Diffuse Horizontal Illuminance'] = df1['DHR'].values.flatten()*179

ithaca_2019.dataframe['Zenith Luminance'] = np.array(
    np.zeros(8760)).astype(int)

ithaca_2019.dataframe['Wind Direction'] = df1['Wind Direction'].values.flatten(
).astype(int)
ithaca_2019.dataframe['Wind Speed'] = df1['Wind Speed'].values.flatten()

#not sure which one to use (skycover)
ithaca_2019.dataframe['Total Sky Cover'] = df1['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL1_2019 FILE 08.27.2020)
ithaca_2019.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)'] = df1['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#

ithaca_2019.dataframe['Visibility'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Ceiling Height'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Present Weather Observation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Present Weather Codes'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Precipitable Water'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Aerosol Optical Depth'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Snow Depth'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Days Since Last Snowfall'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Albedo'] = np.array(np.zeros(8760)).astype(int)
#

# We have precipitation but lets not use it for now
ithaca_2019.dataframe['Liquid Precipitation Depth'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Liquid Precipitation Quantity'] = np.array(
    np.zeros(8760)).astype(int)

pd.options.display.max_columns = None
display(ithaca_2019.dataframe.loc[8:20])


ithaca_2019.write('TjadenHallStation2019.epw')




,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover (used if Horizontal IR Intensity missing),Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
8,2019,1,1,9,0,0,-0.381019,-4.259259,75.033333,28.873917,0,0,0,4.600000,0.242680,4.462000,823.400000,43.439808,798.698000,0,83,1.500000,10,10,0,0,0,0,0,0,0,0,0,0,0
9,2019,1,1,10,0,0,-0.306296,-4.231481,74.933333,28.905417,0,0,0,15.216667,0.775458,14.760167,2723.783334,138.807037,2642.069834,0,63,1.500000,10,10,0,0,0,0,0,0,0,0,0,0,0
10,2019,1,1,11,0,0,0.120370,-4.000000,73.800000,28.916733,0,0,0,61.883333,3.123057,60.026833,11077.116666,559.027276,10744.803166,0,82,1.870000,10,10,0,0,0,0,0,0,0,0,0,0,0
11,2019,1,1,12,0,0,0.687222,-4.092593,70.183333,28.908383,0,0,0,80.200000,4.108942,77.794000,14355.800000,735.500615,13925.126000,0,110,2.366667,10,10,0,0,0,0,0,0,0,0,0,0,0
12,2019,1,1,13,0,0,0.668981,-4.083333,70.433333,28.902467,0,0,0,75.200000,4.009965,72.944000,13460.800000,717.783668,13056.976000,0,92,2.240000,10,10,0,0,0,0,0,0,0,0,0,0,0
13,2019,1,1,14,0,0,0.222500,-3.574074,75.550000,28.904933,0,0,0,33.750000,1.920696,32.737500,6041.250000,343.804599,5860.012500,0,124,2.116667,10,10,0,0,0,0,0,0,0,0,0,0,0
14,2019,1,1,15,0,0,0.028426,-3.074074,79.600000,28.920700,0,0,0,13.733333,0.855428,13.321333,2458.266666,153.121616,2384.518666,0,87,1.623333,10,10,0,0,0,0,0,0,0,0,0,0,0
15,2019,1,1,16,0,0,0.185741,-3.481481,76.500000,28.943333,0,0,0,6.466667,0.452074,6.272667,1157.533333,80.921193,1122.807333,0,100,1.993333,10,10,0,0,0,0,0,0,0,0,0,0,0
16,2019,1,1,17,0,0,0.120926,-4.222222,72.733333,28.976300,0,0,0,1.033333,0.083056,1.002333,184.966667,14.867103,179.417667,0,115,2.366667,10,10,0,0,0,0,0,0,0,0,0,0,0
17,2019,1,1,18,0,0,0.060463,-4.481481,71.483333,28.996000,0,0,0,0.600000,0.056647,0.582000,107.400000,10.139756,104.178000,0,101,1.746667,10,10,0,0,0,0,0,0,0,0,0,0,0


In [45]:
import time
from tqdm import tqdm

S2 = HoboReader("DL2_PSB_20202021_EPW.csv")
df2 = S2.get_dataframe()

def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0

    i = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        GHR = df.iloc[i, df.columns.get_loc('Solar Radiation')].values[0]
        DNI, DHR = GenReindl(lat, lon, time_zone).calc_split(df.month[i], df.day[i], df.hour[i], GHR)
        #print(i,  GHR, DNI, DHR)
        df.iloc[i, df.columns.get_loc('DHR')] = DHR
        df.iloc[i, df.columns.get_loc('DNI')] = DNI
        i += 1

    return df


df2 = calc_split_df(df2)



  0%|                                                                                         | 0/8760 [00:00<?, ?it/s]

  0%|                                                                                 | 7/8760 [00:00<02:14, 64.97it/s]

  0%|▏                                                                               | 15/8760 [00:00<02:10, 66.84it/s]

  0%|▏                                                                               | 23/8760 [00:00<02:06, 69.08it/s]

  0%|▎                                                                               | 30/8760 [00:00<02:06, 69.21it/s]

  0%|▎                                                                               | 38/8760 [00:00<02:02, 71.03it/s]

  1%|▍                                                                               | 45/8760 [00:00<02:03, 70.56it/s]

  1%|▍                                                                               | 54/8760 [00:00<01:58, 73.47it/s]

  1%|▌                        

  6%|████▊                                                                          | 531/8760 [00:07<02:22, 57.71it/s]

  6%|████▊                                                                          | 538/8760 [00:07<02:19, 58.81it/s]

  6%|████▉                                                                          | 544/8760 [00:07<02:30, 54.54it/s]

  6%|████▉                                                                          | 552/8760 [00:07<02:17, 59.53it/s]

  6%|█████                                                                          | 560/8760 [00:08<02:10, 63.00it/s]

  6%|█████                                                                          | 567/8760 [00:08<02:23, 56.93it/s]

  7%|█████▏                                                                         | 573/8760 [00:08<02:44, 49.81it/s]

  7%|█████▏                                                                         | 579/8760 [00:08<03:07, 43.61it/s]

  7%|█████▎                     

 12%|█████████                                                                     | 1019/8760 [00:15<01:41, 76.59it/s]

 12%|█████████▏                                                                    | 1028/8760 [00:15<01:38, 78.14it/s]

 12%|█████████▏                                                                    | 1036/8760 [00:15<01:39, 77.83it/s]

 12%|█████████▎                                                                    | 1045/8760 [00:15<01:38, 78.63it/s]

 12%|█████████▍                                                                    | 1054/8760 [00:15<01:37, 79.21it/s]

 12%|█████████▍                                                                    | 1062/8760 [00:15<01:37, 79.04it/s]

 12%|█████████▌                                                                    | 1070/8760 [00:15<01:37, 79.15it/s]

 12%|█████████▌                                                                    | 1079/8760 [00:16<01:36, 79.67it/s]

 12%|█████████▋                 

 18%|██████████████                                                                | 1584/8760 [00:22<01:31, 78.78it/s]

 18%|██████████████▏                                                               | 1593/8760 [00:22<01:29, 79.73it/s]

 18%|██████████████▎                                                               | 1601/8760 [00:22<01:35, 74.95it/s]

 18%|██████████████▎                                                               | 1609/8760 [00:22<01:33, 76.24it/s]

 18%|██████████████▍                                                               | 1617/8760 [00:22<01:33, 76.50it/s]

 19%|██████████████▍                                                               | 1626/8760 [00:23<01:31, 77.88it/s]

 19%|██████████████▌                                                               | 1634/8760 [00:23<01:30, 78.33it/s]

 19%|██████████████▌                                                               | 1642/8760 [00:23<01:30, 78.66it/s]

 19%|██████████████▋            

 24%|███████████████████                                                           | 2144/8760 [00:29<01:32, 71.85it/s]

 25%|███████████████████▏                                                          | 2152/8760 [00:30<01:39, 66.27it/s]

 25%|███████████████████▏                                                          | 2160/8760 [00:30<01:35, 68.76it/s]

 25%|███████████████████▎                                                          | 2168/8760 [00:30<01:33, 70.69it/s]

 25%|███████████████████▍                                                          | 2176/8760 [00:30<01:30, 72.71it/s]

 25%|███████████████████▍                                                          | 2184/8760 [00:30<01:28, 74.39it/s]

 25%|███████████████████▌                                                          | 2192/8760 [00:30<01:27, 74.99it/s]

 25%|███████████████████▌                                                          | 2201/8760 [00:30<01:25, 76.58it/s]

 25%|███████████████████▋       

...

 71%|███████████████████████████████████████████████████████▎                      | 6207/8760 [01:20<00:32, 77.96it/s]

 71%|███████████████████████████████████████████████████████▎                      | 6216/8760 [01:21<00:32, 78.93it/s]

 71%|███████████████████████████████████████████████████████▍                      | 6225/8760 [01:21<00:31, 79.63it/s]

 71%|███████████████████████████████████████████████████████▍                      | 6233/8760 [01:21<00:32, 77.71it/s]

 71%|███████████████████████████████████████████████████████▌                      | 6241/8760 [01:21<00:33, 75.36it/s]

 71%|███████████████████████████████████████████████████████▋                      | 6249/8760 [01:21<00:33, 74.20it/s]

 71%|███████████████████████████████████████████████████████▋                      | 6257/8760 [01:21<00:34, 73.40it/s]

 72%|███████████████████████████████████████████████████████▊                      | 6265/8760 [01:21<00:34, 71.69it/s]

 72%|███████████████████████████

 77%|████████████████████████████████████████████████████████████▍                 | 6782/8760 [01:28<00:24, 79.88it/s]

 78%|████████████████████████████████████████████████████████████▍                 | 6791/8760 [01:28<00:25, 76.82it/s]

 78%|████████████████████████████████████████████████████████████▌                 | 6799/8760 [01:28<00:26, 74.55it/s]

 78%|████████████████████████████████████████████████████████████▌                 | 6808/8760 [01:28<00:25, 76.45it/s]

 78%|████████████████████████████████████████████████████████████▋                 | 6816/8760 [01:28<00:25, 76.43it/s]

 78%|████████████████████████████████████████████████████████████▊                 | 6824/8760 [01:28<00:26, 74.10it/s]

 78%|████████████████████████████████████████████████████████████▊                 | 6832/8760 [01:29<00:25, 75.19it/s]

 78%|████████████████████████████████████████████████████████████▉                 | 6841/8760 [01:29<00:24, 77.32it/s]

 78%|███████████████████████████

 84%|█████████████████████████████████████████████████████████████████▌            | 7365/8760 [01:35<00:16, 85.92it/s]

 84%|█████████████████████████████████████████████████████████████████▋            | 7374/8760 [01:35<00:16, 85.93it/s]

 84%|█████████████████████████████████████████████████████████████████▋            | 7383/8760 [01:35<00:15, 86.18it/s]

 84%|█████████████████████████████████████████████████████████████████▊            | 7392/8760 [01:35<00:15, 85.86it/s]

 84%|█████████████████████████████████████████████████████████████████▉            | 7401/8760 [01:36<00:15, 86.13it/s]

 85%|█████████████████████████████████████████████████████████████████▉            | 7410/8760 [01:36<00:15, 84.86it/s]

 85%|██████████████████████████████████████████████████████████████████            | 7419/8760 [01:36<00:15, 85.91it/s]

 85%|██████████████████████████████████████████████████████████████████▏           | 7428/8760 [01:36<00:15, 86.42it/s]

 85%|███████████████████████████

 91%|██████████████████████████████████████████████████████████████████████▉       | 7962/8760 [01:42<00:09, 79.95it/s]

 91%|██████████████████████████████████████████████████████████████████████▉       | 7971/8760 [01:42<00:09, 81.89it/s]

 91%|███████████████████████████████████████████████████████████████████████       | 7980/8760 [01:43<00:09, 83.29it/s]

 91%|███████████████████████████████████████████████████████████████████████▏      | 7989/8760 [01:43<00:09, 81.79it/s]

 91%|███████████████████████████████████████████████████████████████████████▏      | 7998/8760 [01:43<00:09, 79.48it/s]

 91%|███████████████████████████████████████████████████████████████████████▎      | 8006/8760 [01:43<00:09, 78.52it/s]

 91%|███████████████████████████████████████████████████████████████████████▎      | 8014/8760 [01:43<00:09, 78.57it/s]

 92%|███████████████████████████████████████████████████████████████████████▍      | 8022/8760 [01:43<00:09, 78.58it/s]

 92%|███████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▊  | 8513/8760 [01:50<00:03, 77.55it/s]

 97%|███████████████████████████████████████████████████████████████████████████▊  | 8521/8760 [01:50<00:03, 78.10it/s]

 97%|███████████████████████████████████████████████████████████████████████████▉  | 8529/8760 [01:50<00:03, 74.34it/s]

 97%|████████████████████████████████████████████████████████████████████████████  | 8537/8760 [01:50<00:03, 69.50it/s]

 98%|████████████████████████████████████████████████████████████████████████████  | 8545/8760 [01:50<00:03, 66.79it/s]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 8552/8760 [01:51<00:03, 65.14it/s]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 8559/8760 [01:51<00:03, 65.63it/s]

 98%|████████████████████████████████████████████████████████████████████████████▎ | 8566/8760 [01:51<00:03, 64.01it/s]

 98%|███████████████████████████

# Variables in the epw that we need


# From GFR
average temperature (F),
 relative humidity (%), 
solar radiation (langleys), 
10 meter wind speed (mph), 
wind direction (degrees), 
precipitation (inches)

# To EPW

average temperature (°C),
relative humidity (%), 
GHI  (Wh/m2), 
DNI  (Wh/m2), 
DHI  (Wh/m2), 
10 meter wind speed (m/s), 
wind direction (degrees), 
precipitation (cm or mm)


# Make the 2019 file
# Make the 2020 file and replace half of the year with the values from 2019

 df5_2020[start_time:end_time] = df5_2019.loc[start_time:end_time]



In [46]:
# GAME FARM ROAD_ITHACA STATION

import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

#S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
#df5 = S5.get_dataframe()

#There are about 35 variables in the core weather data. However, not all of them are used by EnergyPlus. Actually, despite of date and time columns, only 13 columns are used:
#dry bulb temperature 
#dew point temperature 
#relative humidity 
#atmospheric pressure 
#horizontal infrared radiation intensity from sky 
#direct normal radiation 
#diffuse horizontal radiation 
#wind direction 
#wind speed 
#present weather observation 
#present weather codes 
#snow depth 
#liquid precipitation depth

# https://www.radiance-online.org/pipermail/radiance-general/2015-April/010841.html

# Daniel,

# You should be able to generate the .wea file from diffuse horizontal
# radiation and direct normal radiation values. The structure of the lines
# after the heading in .wea file is:

# month day hour directNormalRadiation diffuseHorizontalRadiation

# Template

template = epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns


#Load NREL 2019 and take atmospheric pressure from there
nrel_2019 = Path.cwd() /  Path( "nrel_data/1162143_42.45_-76.50_2019.csv")
#print(nrel_2019)
nrel_2019_df = pd.read_csv(nrel_2019)
#df.columns = df.index[0]
nrel_2019_df.columns = nrel_2019_df.loc[nrel_2019_df.index[1]]
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])


nrel_2019_pressure = nrel_2019_df['Pressure'].astype(int)*100




# Real file

ithaca_2019 = epw()
# ithaca_2019.headers['LOCATION'][0]='Ithaca_NY'

#Design conditions and extreme periods are taken from the Syracuse file

ithaca_2019.headers = {'LOCATION': ['Ithaca',
                                    'NY',
                                    'USA',
                                    'TMY3',
                                    '725190',
                                    '42.438934',
                                    '-76.492435',
                                    '-5.0',
                                    '125.0'],
                       'DESIGN CONDITIONS': ['1',
  'Climate Design Data 2009 ASHRAE Handbook',
  '',
  'Heating',
  '1',
  '-19.3',
  '-16.2',
  '-23.6',
  '0.5',
  '-18.1',
  '-20.6',
  '0.6',
  '-15.1',
  '12.8',
  '-5.9',
  '11.9',
  '-5.9',
  '3',
  '90',
  'Cooling',
  '7',
  '10.8',
  '31.6',
  '22.8',
  '30',
  '21.8',
  '28.5',
  '21',
  '24.1',
  '29.3',
  '23.1',
  '27.9',
  '22.3',
  '26.8',
  '4.4',
  '260',
  '22.5',
  '17.4',
  '26.9',
  '21.5',
  '16.4',
  '25.8',
  '20.7',
  '15.6',
  '25.1',
  '72.9',
  '29.6',
  '69.2',
  '27.9',
  '65.9',
  '26.8',
  '695',
  'Extremes',
  '10.8',
  '9.1',
  '8.2',
  '27.6',
  '-23.6',
  '33.8',
  '4',
  '2',
  '-26.5',
  '35.2',
  '-28.8',
  '36.3',
  '-31.1',
  '37.4',
  '-34',
  '38.9'],
 'TYPICAL/EXTREME PERIODS': ['6',
  'Summer - Week Nearest Max Temperature For Period',
  'Extreme',
  '7/27',
  '8/ 2',
  'Summer - Week Nearest Average Temperature For Period',
  'Typical',
  '8/24',
  '8/30',
  'Winter - Week Nearest Min Temperature For Period',
  'Extreme',
  '1/27',
  '2/ 2',
  'Winter - Week Nearest Average Temperature For Period',
  'Typical',
  '12/22',
  '12/28',
  'Autumn - Week Nearest Average Temperature For Period',
  'Typical',
  '10/20',
  '10/26',
  'Spring - Week Nearest Average Temperature For Period',
  'Typical',
  '3/15',
  '3/21'],
                       'GROUND TEMPERATURES': ['3',
                                               '0.5',
                                               '',
                                               '',
                                               '',
                                               '2.44',
                                               '4.62',
                                               '9.34',
                                               '15.56',
                                               '21.51',
                                               '25.58',
                                               '26.67',
                                               '24.44',
                                               '19.55',
                                               '13.31',
                                               '7.40',
                                               '3.42',
                                               '2',
                                               '',
                                               '',
                                               '',
                                               '5.55',
                                               '6.08',
                                               '8.78',
                                               '13.07',
                                               '17.76',
                                               '21.58',
                                               '23.50',
                                               '23.98',
                                               '20.18',
                                               '15.85',
                                               '11.17',
                                               '7.40',
                                               '4',
                                               '',
                                               '',
                                               '',
                                               '8.79',
                                               '8.41',
                                               '9.60',
                                               '12.12',
                                               '15.30',
                                               '18.27',
                                               '20.23',
                                               '20.66',
                                               '19.42',
                                               '16.85',
                                               '13.67',
                                               '10.73'],
                       'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
                       'COMMENTS 1': [''],
                       'COMMENTS 2': [''],
                       'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

#print(ithaca_2019.headers)


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

ithaca_2019.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
ithaca_2019.dataframe['Month'] = template.dataframe['Month'].astype(int)
ithaca_2019.dataframe['Day'] = template.dataframe['Day'].astype(int)
ithaca_2019.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
ithaca_2019.dataframe['Minute'] = template.dataframe['Minute'].astype(int)
ithaca_2019.dataframe['Data Source and Uncertainty Flags'] = np.array(
    np.zeros(8760)).astype(int)


# Actual file starts here
# 6
# .apply(lambda x: ftoc(x), axis=1).values.flatten()
ithaca_2019.dataframe['Dry Bulb Temperature'] = df5['Temp'].values.flatten()

#     Add this later (ADDED TO DL5_2019 FILE 07.22.2020)
# np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Dew Point Temperature'] = df5['DewPoint'].values.flatten()


ithaca_2019.dataframe['Relative Humidity'] = df5['RH'].values.flatten()

# Added NREL pressure 2020-08-21
ithaca_2019.dataframe['Atmospheric Station Pressure'] = nrel_2019_pressure.values.flatten()
ithaca_2019.dataframe['Extraterrestrial Horizontal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Extraterrestrial Direct Normal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Horizontal Infrared Radiation Intensity'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Global Horizontal Radiation'] = df5['Solar Radiation'].values.flatten()
ithaca_2019.dataframe['Direct Normal Radiation'] = df5['DNI'].values.flatten()
ithaca_2019.dataframe['Diffuse Horizontal Radiation'] = df5['DHR'].values.flatten()

# Do we need this?
# https://www.radiance-online.org//pipermail/radiance-general/2013-January/009140.html ?
ithaca_2019.dataframe['Global Horizontal Illuminance'] = df5['Solar Radiation'].values.flatten()*179
ithaca_2019.dataframe['Direct Normal Illuminance'] = df5['DNI'].values.flatten()*179
ithaca_2019.dataframe['Diffuse Horizontal Illuminance'] = df5['DHR'].values.flatten()*179

ithaca_2019.dataframe['Zenith Luminance'] = np.array(
    np.zeros(8760)).astype(int)

ithaca_2019.dataframe['Wind Direction'] = df5['Wind Direction'].values.flatten(
).astype(int)
ithaca_2019.dataframe['Wind Speed'] = df5['Wind Speed'].values.flatten()

#not sure which one to use (skycover)
ithaca_2019.dataframe['Total Sky Cover'] = df5['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020)
ithaca_2019.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)'] = df5['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#

ithaca_2019.dataframe['Visibility'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Ceiling Height'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Present Weather Observation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Present Weather Codes'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Precipitable Water'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Aerosol Optical Depth'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Snow Depth'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Days Since Last Snowfall'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Albedo'] = np.array(np.zeros(8760)).astype(int)
#

# We have precipitation but lets not use it for now
ithaca_2019.dataframe['Liquid Precipitation Depth'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Liquid Precipitation Quantity'] = np.array(
    np.zeros(8760)).astype(int)

pd.options.display.max_columns = None
display(ithaca_2019.dataframe.loc[8:20])


ithaca_2019.write('GameFarmRoad2019.epw')




ValueError: Length of values does not match length of index

In [ ]:
# PSB Parking Lot

import math
import pandas as pd
from hoboreader import HoboReader
S2 = HoboReader("DL2_PSB_20202021_EPW.csv")
df2 = S2.get_dataframe()


df2 = calc_split_df(df2)



  0%|                                                                                         | 0/8760 [00:00<?, ?it/s]

  0%|                                                                                 | 7/8760 [00:00<02:15, 64.38it/s]

  0%|▏                                                                               | 16/8760 [00:00<02:07, 68.82it/s]

  0%|▏                                                                               | 24/8760 [00:00<02:03, 70.74it/s]

  0%|▎                                                                               | 33/8760 [00:00<01:57, 73.98it/s]

  0%|▎                                                                               | 41/8760 [00:00<01:58, 73.66it/s]

  1%|▍                                                                               | 49/8760 [00:00<01:57, 74.05it/s]

  1%|▌                                                                               | 57/8760 [00:00<01:55, 75.36it/s]

  1%|▌                        

  6%|█████                                                                          | 562/8760 [00:07<01:46, 76.87it/s]

  7%|█████▏                                                                         | 571/8760 [00:07<01:44, 78.56it/s]

  7%|█████▏                                                                         | 579/8760 [00:07<01:43, 78.81it/s]

  7%|█████▎                                                                         | 588/8760 [00:07<01:42, 79.96it/s]

  7%|█████▍                                                                         | 597/8760 [00:07<01:42, 79.72it/s]

  7%|█████▍                                                                         | 606/8760 [00:07<01:41, 79.98it/s]

  7%|█████▌                                                                         | 615/8760 [00:07<01:40, 80.80it/s]

  7%|█████▋                                                                         | 624/8760 [00:07<01:44, 77.61it/s]

  7%|█████▋                     

 13%|█████████▉                                                                    | 1113/8760 [00:14<01:32, 82.37it/s]

 13%|█████████▉                                                                    | 1122/8760 [00:14<01:33, 82.04it/s]

 13%|██████████                                                                    | 1131/8760 [00:14<01:32, 82.04it/s]

 13%|██████████▏                                                                   | 1140/8760 [00:15<01:32, 82.04it/s]

 13%|██████████▏                                                                   | 1149/8760 [00:15<01:33, 81.38it/s]

 13%|██████████▎                                                                   | 1158/8760 [00:15<01:33, 81.63it/s]

 13%|██████████▍                                                                   | 1167/8760 [00:15<01:33, 81.09it/s]

 13%|██████████▍                                                                   | 1176/8760 [00:15<01:37, 77.69it/s]

 14%|██████████▌                

 19%|██████████████▊                                                               | 1665/8760 [00:21<01:33, 75.63it/s]

 19%|██████████████▉                                                               | 1673/8760 [00:22<01:33, 75.87it/s]

 19%|██████████████▉                                                               | 1681/8760 [00:22<01:33, 75.81it/s]

 19%|███████████████                                                               | 1689/8760 [00:22<01:33, 75.87it/s]

 19%|███████████████                                                               | 1697/8760 [00:22<01:32, 76.68it/s]

 19%|███████████████▏                                                              | 1706/8760 [00:22<01:30, 77.60it/s]

 20%|███████████████▎                                                              | 1715/8760 [00:22<01:29, 78.47it/s]

 20%|███████████████▎                                                              | 1724/8760 [00:22<01:28, 79.51it/s]

 20%|███████████████▍           

 25%|███████████████████▋                                                          | 2211/8760 [00:29<01:34, 69.24it/s]

 25%|███████████████████▊                                                          | 2219/8760 [00:29<01:32, 71.05it/s]

 25%|███████████████████▊                                                          | 2227/8760 [00:29<01:29, 72.97it/s]

 26%|███████████████████▉                                                          | 2235/8760 [00:29<01:27, 74.17it/s]

 26%|███████████████████▉                                                          | 2243/8760 [00:29<01:28, 73.79it/s]

 26%|████████████████████                                                          | 2251/8760 [00:29<01:27, 74.35it/s]

 26%|████████████████████                                                          | 2259/8760 [00:29<01:25, 75.80it/s]

 26%|████████████████████▏                                                         | 2267/8760 [00:30<01:31, 70.75it/s]

 26%|████████████████████▎      

 31%|████████████████████████▌                                                     | 2759/8760 [00:36<01:11, 84.06it/s]

 32%|████████████████████████▋                                                     | 2768/8760 [00:36<01:13, 81.41it/s]

 32%|████████████████████████▋                                                     | 2777/8760 [00:37<01:16, 78.20it/s]

 32%|████████████████████████▊                                                     | 2785/8760 [00:37<01:16, 77.66it/s]

 32%|████████████████████████▊                                                     | 2793/8760 [00:37<01:21, 72.85it/s]

 32%|████████████████████████▉                                                     | 2801/8760 [00:37<01:20, 73.88it/s]

 32%|█████████████████████████                                                     | 2810/8760 [00:37<01:17, 77.13it/s]

 32%|█████████████████████████                                                     | 2819/8760 [00:37<01:14, 80.00it/s]

 32%|█████████████████████████▏ 

 38%|█████████████████████████████▌                                                | 3324/8760 [00:44<01:09, 77.68it/s]

 38%|█████████████████████████████▋                                                | 3332/8760 [00:44<01:10, 77.06it/s]

 38%|█████████████████████████████▋                                                | 3340/8760 [00:44<01:10, 76.39it/s]

 38%|█████████████████████████████▊                                                | 3348/8760 [00:44<01:14, 72.45it/s]

 38%|█████████████████████████████▉                                                | 3357/8760 [00:44<01:12, 74.90it/s]

 38%|█████████████████████████████▉                                                | 3366/8760 [00:44<01:10, 76.71it/s]

 39%|██████████████████████████████                                                | 3374/8760 [00:44<01:10, 75.96it/s]

 39%|██████████████████████████████                                                | 3382/8760 [00:44<01:11, 75.67it/s]

 39%|███████████████████████████

In [32]:
df2.to_csv("PSB2020_edit.csv")


In [40]:



df2 = pd.read_csv("PSB2020_edit_done.csv")
display(df2)

,Plot Title: Parking Lot_PSB 2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,#,"Date Time, GMT-05:00","Wind Speed, mph (LGR S/N: 20702489, SEN S/N: 2...","Gust Speed, mph (LGR S/N: 20702489, SEN S/N: 2...","Wind Direction, ø (LGR S/N: 20702489, SEN S/N:...","MRT, °F (LGR S/N: 20702489, SEN S/N: 20693834)","Temp, °F (LGR S/N: 20702489, SEN S/N: 20698453)","RH, % (LGR S/N: 20702489, SEN S/N: 20698453)","Solar Radiation, W/m² (LGR S/N: 20702489, SEN ...","DewPt, °F (LGR S/N: 20702489, SEN S/N: 20698453)","SkyCover, Fraction (LGR S/N: 20095520, SEN S/N...",NaN,NaN,NaN
1,2020-01-01 00:00:00-05:00,1.2830048000000003,2.837213866666667,51.4,0.503611111,0.6413888888333333,65.13333333333334,0.6,-5.1944444443333335,0,1,1.0,0.582,0.091845
2,2020-01-01 01:00:00-05:00,0.8940800000000001,2.2255141333333337,52.800000000000004,0.2733333333333334,0.3842592591666667,67.45,0.6,-4.925925926000001,1,1,1.0,0.582,0.077778
3,2020-01-01 02:00:00-05:00,1.1712448000000002,3.5629088,51.86666666666667,0.15768518533333334,0.27324074083333333,68.66666666666667,0.6,-4.814814814833333,2,1,1.0,0.582,0.064993
4,2020-01-01 03:00:00-05:00,1.0043498666666668,2.4453088,46.71666666666666,0.06527777783333334,0.15333333333333332,70.85,0.6,-4.5185185185,3,1,1.0,0.582,0.054637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8780,2020-12-31 19:00:00-05:00,1.5072698666666666,4.062848533333334,47.416666666666664,1.43,1.6198148148333333,62.833333333333336,0.6,-4.712962963,19,31,12.0,0.582,0.079521
8781,2020-12-31 20:00:00-05:00,1.2830048000000003,3.6731786666666664,47.61666666666667,1.0442592591666666,1.2259259258333335,63.75,0.6,-4.8703703705,20,31,12.0,0.582,0.092872
8782,2020-12-31 21:00:00-05:00,1.1146197333333334,3.6746687999999996,50.45000000000001,0.9483333331666667,1.0848148148333334,64.58333333333333,0.6,-4.842592592666667,21,31,12.0,0.582,0.103325
8783,2020-12-31 22:00:00-05:00,1.1131296000000002,3.5077738666666662,52.099999999999994,0.8477777776666667,1.0164814815,64.28333333333335,0.6,-4.9814814815,22,31,12.0,0.582,0.106595


In [47]:
# PSB_ITHACA STATION

import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

S2 = HoboReader("DL2_PSB_20202021_EPW.csv")
df2 = S2.get_dataframe()

# Template

template = epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns

#Load NREL 2019 and take atmospheric pressure from there
nrel_2019 = Path.cwd() /  Path( "nrel_data/1162143_42.45_-76.50_2019.csv")
#print(nrel_2019)
nrel_2019_df = pd.read_csv(nrel_2019)
#df.columns = df.index[0]
nrel_2019_df.columns = nrel_2019_df.loc[nrel_2019_df.index[1]]
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])


nrel_2019_pressure = nrel_2019_df['Pressure'].astype(int)*100

# Real file

PSB_2020 = epw()
# PSB_2019.headers['LOCATION'][0]='Ithaca_NY'


PSB_2019.headers = {'LOCATION': ['PSB Parking Lot, Cornell University',
                                 'NY',
                                 'USA',
                                 'TMY3',
                                 '725190',
                                 '42.450620',
                                 '-76.481201',
                                 '-5.0',
                                 '255.0'],
                    'DESIGN CONDITIONS': ['1',
  'Climate Design Data 2009 ASHRAE Handbook',
  '',
  'Heating',
  '1',
  '-19.3',
  '-16.2',
  '-23.6',
  '0.5',
  '-18.1',
  '-20.6',
  '0.6',
  '-15.1',
  '12.8',
  '-5.9',
  '11.9',
  '-5.9',
  '3',
  '90',
  'Cooling',
  '7',
  '10.8',
  '31.6',
  '22.8',
  '30',
  '21.8',
  '28.5',
  '21',
  '24.1',
  '29.3',
  '23.1',
  '27.9',
  '22.3',
  '26.8',
  '4.4',
  '260',
  '22.5',
  '17.4',
  '26.9',
  '21.5',
  '16.4',
  '25.8',
  '20.7',
  '15.6',
  '25.1',
  '72.9',
  '29.6',
  '69.2',
  '27.9',
  '65.9',
  '26.8',
  '695',
  'Extremes',
  '10.8',
  '9.1',
  '8.2',
  '27.6',
  '-23.6',
  '33.8',
  '4',
  '2',
  '-26.5',
  '35.2',
  '-28.8',
  '36.3',
  '-31.1',
  '37.4',
  '-34',
  '38.9'],
 'TYPICAL/EXTREME PERIODS': ['6',
  'Summer - Week Nearest Max Temperature For Period',
  'Extreme',
  '7/27',
  '8/ 2',
  'Summer - Week Nearest Average Temperature For Period',
  'Typical',
  '8/24',
  '8/30',
  'Winter - Week Nearest Min Temperature For Period',
  'Extreme',
  '1/27',
  '2/ 2',
  'Winter - Week Nearest Average Temperature For Period',
  'Typical',
  '12/22',
  '12/28',
  'Autumn - Week Nearest Average Temperature For Period',
  'Typical',
  '10/20',
  '10/26',
  'Spring - Week Nearest Average Temperature For Period',
  'Typical',
  '3/15',
  '3/21'],
                       'GROUND TEMPERATURES': ['3',
                                               '0.5',
                                               '',
                                               '',
                                               '',
                                               '2.44',
                                               '4.62',
                                               '9.34',
                                               '15.56',
                                               '21.51',
                                               '25.58',
                                               '26.67',
                                               '24.44',
                                               '19.55',
                                               '13.31',
                                               '7.40',
                                               '3.42',
                                               '2',
                                               '',
                                               '',
                                               '',
                                               '5.55',
                                               '6.08',
                                               '8.78',
                                               '13.07',
                                               '17.76',
                                               '21.58',
                                               '23.50',
                                               '23.98',
                                               '20.18',
                                               '15.85',
                                               '11.17',
                                               '7.40',
                                               '4',
                                               '',
                                               '',
                                               '',
                                               '8.79',
                                               '8.41',
                                               '9.60',
                                               '12.12',
                                               '15.30',
                                               '18.27',
                                               '20.23',
                                               '20.66',
                                               '19.42',
                                               '16.85',
                                               '13.67',
                                               '10.73'],                
                    'GROUND TEMPERATURES': ['3',
                                            '0.5',
                                            '',
                                            '',
                                            '',
                                            '2.44',
                                            '4.62',
                                            '9.34',
                                            '15.56',
                                            '21.51',
                                            '25.58',
                                            '26.67',
                                            '24.44',
                                            '19.55',
                                            '13.31',
                                            '7.40',
                                            '3.42',
                                            '2',
                                            '',
                                            '',
                                            '',
                                            '5.55',
                                            '6.08',
                                            '8.78',
                                            '13.07',
                                            '17.76',
                                            '21.58',
                                            '23.50',
                                            '23.98',
                                            '20.18',
                                            '15.85',
                                            '11.17',
                                            '7.40',
                                            '4',
                                            '',
                                            '',
                                            '',
                                            '8.79',
                                            '8.41',
                                            '9.60',
                                            '12.12',
                                            '15.30',
                                            '18.27',
                                            '20.23',
                                            '20.66',
                                            '19.42',
                                            '16.85',
                                            '13.67',
                                            '10.73'],
                    'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
                    'COMMENTS 1': [''],
                    'COMMENTS 2': [''],
                    'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

#print(PSB_2019.headers)


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

PSB_2020.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
PSB_2020.dataframe['Month'] = template.dataframe['Month'].astype(int)
PSB_2020.dataframe['Day'] = template.dataframe['Day'].astype(int)
PSB_2020.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
PSB_2020.dataframe['Minute'] = template.dataframe['Minute'].astype(int)
PSB_2020.dataframe['Data Source and Uncertainty Flags'] = np.array(
    np.zeros(8760)).astype(int)


# Actual file starts here
# 6
# .apply(lambda x: ftoc(x), axis=1).values.flatten()
PSB_2020.dataframe['Dry Bulb Temperature'] = df2['Temp'].values.flatten()
# np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Dew Point Temperature'] = df2['DewPt'].values.flatten()


PSB_2020.dataframe['Relative Humidity'] = df2['RH'].values.flatten()

# Added NREL pressure 2020-08-21
PSB_2020.dataframe['Atmospheric Station Pressure'] =  nrel_2019_pressure.values.flatten()
PSB_2020.dataframe['Extraterrestrial Horizontal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Extraterrestrial Direct Normal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Horizontal Infrared Radiation Intensity'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Global Horizontal Radiation'] = df2['Solar Radiation'].values.flatten()
PSB_2020.dataframe['Direct Normal Radiation'] = df2['DNI'].values.flatten()
PSB_2020.dataframe['Diffuse Horizontal Radiation'] = df2['DHR'].values.flatten()

# Do we need this?
PSB_2020.dataframe['Global Horizontal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Direct Normal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Diffuse Horizontal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Zenith Luminance'] = np.array(np.zeros(8760)).astype(int)

PSB_2020.dataframe['Wind Direction'] = df2['Wind Direction'].values.flatten(
).astype(int)
PSB_2020.dataframe['Wind Speed'] = df2['Wind Speed'].values.flatten()

#not sure which one to use (skycover)
PSB_2020.dataframe['Total Sky Cover'] = df2['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020)
PSB_2020.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)'] = df2['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#

PSB_2020.dataframe['Visibility'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Ceiling Height'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Present Weather Observation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Present Weather Codes'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Precipitable Water'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Aerosol Optical Depth'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Snow Depth'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Days Since Last Snowfall'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Albedo'] = np.array(np.zeros(8760)).astype(int)
#

# We have precipitation but lets not use it for now
PSB_2020.dataframe['Liquid Precipitation Depth'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Liquid Precipitation Quantity'] = np.array(
    np.zeros(8760)).astype(int)

pd.options.display.max_columns = None
display(PSB_2020.dataframe.loc[8:20])



PSB_2020write('PSBLot_20202021.epw')

KeyError: 'DNI'